<a href="https://colab.research.google.com/github/KlymentievMaksym/IntelOb/blob/main/Lab5/Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from tqdm import tqdm

import tensorflow as tf
layers = tf.keras.layers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
# from DE import DE

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2)

In [7]:
filepath_save = "./Models/Model_valAcc{val_accuracy:.2f}_valLoss{val_loss:.2f}.keras"
filepath_backup = "./Backups/"
callbacks = [
    # tf.keras.callbacks.ModelCheckpoint(filepath_save, monitor='val_accuracy', save_best_only=True),
    # tf.keras.callbacks.BackupAndRestore(filepath_backup),
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
]

data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
  layers.RandomContrast(0.2),
])

In [18]:
def DE(pop_size, iterations, function, limits):
    dim = len(limits)
    limits = np.array(limits)
    x_low = limits[:, 0]
    x_high = limits[:, 1]

    population = np.random.uniform(x_low, x_high, (pop_size, dim))

    max_f = -float('inf')
    best_f = float('inf')
    best_pop = np.zeros(dim)

    fitness = np.array([function(X) for X in population])
    for iteration in tqdm(
        range(iterations),
        desc="Processing",
        unit="step",
        bar_format="{l_bar}{bar:40}{r_bar}",
        colour='cyan',
        total=iterations
    ):
        for i in range(pop_size):
            F = np.random.uniform(1e-6, 2)
            P = np.random.uniform(1e-6, 1)
            r = np.random.uniform(1e-6, 1, dim)
            x1, x2, x3 = np.random.choice(population.shape[0], size=3, replace=False)
            while np.all(population[x1] == population[i]) or np.all(population[x2] == population[i]) or np.all(population[x3] == population[i]):
                x1, x2, x3 = np.random.choice(population.shape[0], size=3, replace=False)
            mutant_vector = population[x1] + F * (population[x2] - population[x3])
            mutant_vector[r < P] = population[i][r < P]
            mutant_fitness = function(mutant_vector)
            if fitness[i] > mutant_fitness:
                fitness[i] = mutant_fitness
                population[i] = mutant_vector.copy()
        el_min = np.argmin(fitness)
        if best_f > fitness[el_min]:
            best_f = fitness[el_min]
            best_pop = population[el_min].copy()
        el_max = np.max(fitness)
        if max_f < el_max:
            max_f = el_max

    return best_f, best_pop

def func(X):
    # print(X)
    amount_of_layer_1 = int(round(X[0]))
    amount_of_layer_2 = int(round(X[1]))
    amount_of_layer_3 = int(round(X[2]))
    amount_of_one_dence = int(round(X[3]))
    amount_of_one_filters = int(round(X[4]))
    # print(np.int64(np.round(X)))
    # print(amount_of_layer_1, amount_of_layer_2, amount_of_layer_3, amount_of_one_dence, amount_of_one_filters)
    model_layers = [
        layers.Input(shape=(32, 32, 3)),
        data_augmentation
    ]

    for _ in range(amount_of_layer_1):
        model_layers.append(layers.Conv2D(amount_of_one_filters, (3, 3), activation='relu', padding='same'))
        model_layers.append(layers.BatchNormalization())
    model_layers.append(layers.MaxPooling2D(2, 2))

    for _ in range(amount_of_layer_2):
        model_layers.append(layers.Conv2D(amount_of_one_filters * 2, (3, 3), activation='relu', padding='same'))
        model_layers.append(layers.BatchNormalization())
    model_layers.append(layers.MaxPooling2D(2, 2))
    model_layers.append(layers.Dropout(0.3))

    for _ in range(amount_of_layer_3):
        model_layers.append(layers.Conv2D(amount_of_one_filters * 4, (3, 3), activation='relu', padding='same'))
        model_layers.append(layers.BatchNormalization())
    model_layers.append(layers.MaxPooling2D(4, 4))
    model_layers.append(layers.Dropout(0.4))

    model_layers.append(layers.Flatten())
    for _ in range(amount_of_one_dence):
        model_layers.append(layers.Dense(amount_of_one_filters * 8, activation='relu'))
    model_layers.append(layers.Dropout(0.5))
    model_layers.append(layers.Dense(10, activation='softmax'))

    model = tf.keras.models.Sequential(model_layers)
    # model.summary()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=2, name="Top2")])
    history = model.fit(x_train, y_train, batch_size=128, epochs=100, validation_data=(x_val, y_val), callbacks=callbacks)
    index = np.argmax(history.history['val_accuracy'])
    return (1.0 - history.history['val_accuracy'][index]) + history.history['val_loss'][index]

func_limits = [[0, 4], [0, 4], [0, 4], [0, 4], [4, 32]]


In [19]:
best_f, best_d = DE(4, 4, func, func_limits)
best_f, best_d

Epoch 1/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - Top2: 0.2127 - accuracy: 0.1099 - loss: 3.8215 - val_Top2: 0.2617 - val_accuracy: 0.1384 - val_loss: 2.2863
Epoch 2/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - Top2: 0.2448 - accuracy: 0.1280 - loss: 2.2897 - val_Top2: 0.3091 - val_accuracy: 0.1677 - val_loss: 2.1864
Epoch 3/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - Top2: 0.3370 - accuracy: 0.1719 - loss: 2.1638 - val_Top2: 0.4364 - val_accuracy: 0.2208 - val_loss: 2.0175
Epoch 4/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - Top2: 0.3770 - accuracy: 0.1906 - loss: 2.1023 - val_Top2: 0.4050 - val_accuracy: 0.2117 - val_loss: 2.0692
Epoch 5/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - Top2: 0.3911 - accuracy: 0.2075 - loss: 2.0759 - val_Top2: 0.4397 - val_accuracy: 0.2317 - val_loss: 2.0044
Epoch 6/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - Top2: 0.4055 - accuracy: 0.2115 - loss: 2.0505 - val_Top2: 0.4652 - val_accuracy: 0.2348 - val_loss: 1.9797
Epoch 7/100
313

Processing:   0%|                                        | 0/4 [00:00<?, ?step/s]

Epoch 1/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 13s 23ms/step - Top2: 0.2815 - accuracy: 0.1509 - loss: 2.6578 - val_Top2: 0.2794 - val_accuracy: 0.1324 - val_loss: 2.4394
Epoch 2/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - Top2: 0.4456 - accuracy: 0.2407 - loss: 2.0139 - val_Top2: 0.5377 - val_accuracy: 0.3021 - val_loss: 1.8116
Epoch 3/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - Top2: 0.5025 - accuracy: 0.2765 - loss: 1.9122 - val_Top2: 0.5292 - val_accuracy: 0.2934 - val_loss: 1.9151
Epoch 4/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - Top2: 0.5386 - accuracy: 0.3057 - loss: 1.8531 - val_Top2: 0.5437 - val_accuracy: 0.3241 - val_loss: 1.8157
Epoch 5/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - Top2: 0.5646 - accuracy: 0.3208 - loss: 1.7961 - val_Top2: 0.6134 - val_accuracy: 0.3797 - val_loss: 1.6604
Epoch 6/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - Top2: 0.5799 - accuracy: 0.3353 - loss: 1.7586 - val_Top2: 0.5905 - val_accuracy: 0.3504 - val_loss: 1.7126
Epoch 7/100
3

Processing:  25%|██████████                              | 1/4 [14:41<44:05, 881.70s/step]

Epoch 1/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 13s 21ms/step - Top2: 0.2806 - accuracy: 0.1472 - loss: 2.5210 - val_Top2: 0.2362 - val_accuracy: 0.1147 - val_loss: 2.4099
Epoch 2/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - Top2: 0.4541 - accuracy: 0.2411 - loss: 2.0043 - val_Top2: 0.5505 - val_accuracy: 0.3160 - val_loss: 1.8131
Epoch 3/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - Top2: 0.5167 - accuracy: 0.2845 - loss: 1.9025 - val_Top2: 0.5359 - val_accuracy: 0.3082 - val_loss: 1.8227
Epoch 4/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - Top2: 0.5469 - accuracy: 0.3163 - loss: 1.8445 - val_Top2: 0.5589 - val_accuracy: 0.3123 - val_loss: 1.8161
Epoch 5/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - Top2: 0.5724 - accuracy: 0.3378 - loss: 1.8053 - val_Top2: 0.6002 - val_accuracy: 0.3551 - val_loss: 1.7074
Epoch 6/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - Top2: 0.5828 - accuracy: 0.3456 - loss: 1.7731 - val_Top2: 0.6271 - val_accuracy: 0.3793 - val_loss: 1.6566
Epoch 7/100


Processing:  50%|████████████████████                    | 2/4 [29:30<29:31, 885.61s/step]

Epoch 1/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 14s 24ms/step - Top2: 0.2467 - accuracy: 0.1258 - loss: 2.6804 - val_Top2: 0.3457 - val_accuracy: 0.1856 - val_loss: 2.1846
Epoch 2/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step - Top2: 0.3915 - accuracy: 0.2044 - loss: 2.0852 - val_Top2: 0.5128 - val_accuracy: 0.2837 - val_loss: 1.8671
Epoch 3/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 20ms/step - Top2: 0.4493 - accuracy: 0.2389 - loss: 2.0075 - val_Top2: 0.5087 - val_accuracy: 0.2767 - val_loss: 1.8642
Epoch 4/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - Top2: 0.4809 - accuracy: 0.2626 - loss: 1.9565 - val_Top2: 0.4646 - val_accuracy: 0.2667 - val_loss: 1.9941
Epoch 5/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - Top2: 0.5027 - accuracy: 0.2738 - loss: 1.9118 - val_Top2: 0.5330 - val_accuracy: 0.3067 - val_loss: 1.8163
Epoch 6/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - Top2: 0.5225 - accuracy: 0.2891 - loss: 1.8818 - val_Top2: 0.5571 - val_accuracy: 0.3037 - val_loss: 1.7752
Epoch 7/100

Processing:  75%|██████████████████████████████          | 3/4 [47:22<16:10, 970.83s/step]

Epoch 1/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 19ms/step - Top2: 0.2650 - accuracy: 0.1391 - loss: 2.5385 - val_Top2: 0.3687 - val_accuracy: 0.1928 - val_loss: 2.1209
Epoch 2/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 16ms/step - Top2: 0.4157 - accuracy: 0.2144 - loss: 2.0413 - val_Top2: 0.4780 - val_accuracy: 0.2660 - val_loss: 1.9151
Epoch 3/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - Top2: 0.4704 - accuracy: 0.2553 - loss: 1.9562 - val_Top2: 0.5608 - val_accuracy: 0.3318 - val_loss: 1.7695
Epoch 4/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - Top2: 0.5177 - accuracy: 0.2963 - loss: 1.8822 - val_Top2: 0.4855 - val_accuracy: 0.2844 - val_loss: 2.0205
Epoch 5/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - Top2: 0.5470 - accuracy: 0.3159 - loss: 1.8303 - val_Top2: 0.4973 - val_accuracy: 0.2881 - val_loss: 1.9093
Epoch 6/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - Top2: 0.5618 - accuracy: 0.3343 - loss: 1.7980 - val_Top2: 0.5821 - val_accuracy: 0.3737 - val_loss: 1.7095
Epoch 7/100
31

Processing: 100%|████████████████████████████████████████| 4/4 [1:06:59<00:00, 1004.84s/step]


In [1]:
best_f, best_d

NameError: name 'best_f' is not defined

In [ ]:
# model = tf.keras.models.Sequential(
#     [
#         layers.Input(shape=(32, 32, 3)),

#         data_augmentation,

#         layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
#         layers.BatchNormalization(),
#         layers.MaxPooling2D(2, 2),

#         layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
#         layers.BatchNormalization(),
#         layers.MaxPooling2D(2, 2),
#         layers.Dropout(0.3),

#         layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
#         layers.BatchNormalization(),
#         layers.MaxPooling2D(4, 4),
#         layers.Dropout(0.4),

#         layers.Flatten(),
#         layers.Dense(1024, activation='relu'),
#         layers.Dropout(0.5),
#         layers.Dense(y_test.shape[1], activation='softmax')
#     ]
# )

# model.summary()
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
#               loss=tf.keras.losses.CategoricalCrossentropy(),
#               metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=2, name="Top2")])

In [ ]:
# history = model.fit(x_train, y_train, batch_size=128, epochs=100, validation_data=(x_val, y_val), callbacks=callbacks)